In [1]:
import pdblp
from xbbg import blp
import workdays
import datetime
import numpy as np
import pandas as pd

T = ["BPITTO01 Index","SBWGNJYC Index","SBUSJYC Index","TPXDDVD Index","TPXDREIT Index","M0KO Index","SREITJWJ Index"]
T += ["USDJPYCR CMPN Curncy","EURJPYCR CMPN Curncy"]
L = ["SBJ","SBF","SBU","EQJ","REJ","EQF","REF","USD","EUR"]

d_from = workdays.workday(datetime.datetime.today(), days=-260).strftime("%Y%m%d")
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime("%Y%m%d")
df = blp.bdh(T, ["px_last"], d_from, calendar="5D", Fill="P").reset_index()

df = df[["index"]+T]
df.columns = ["Date"] + L
df.insert(loc = 1, column= "ME", value= 0) #MonthEnd
df

,Date,ME,SBJ,SBF,SBU,EQJ,REJ,EQF,REF,USD,EUR
0,2022-09-26,0,373.749,350.2316,329.2774,3018.01,4553.78,5229.99,1328.35,202.1539,143.2909
1,2022-09-27,0,372.637,347.0810,327.1475,3032.15,4487.65,5232.80,1307.69,202.2518,143.1238
2,2022-09-28,0,372.435,351.5094,332.0198,3003.25,4414.36,5311.99,1331.00,201.3854,144.5921
3,2022-09-29,0,373.426,350.2613,330.9340,3057.13,4464.91,5224.31,1301.06,201.8334,146.1065
4,2022-09-30,0,373.602,350.4020,329.8045,3003.39,4488.86,5183.99,1322.31,202.2546,146.1895
...,...,...,...,...,...,...,...,...,...,...,...
255,2023-09-18,0,NaN,335.8675,314.1616,NaN,NaN,6690.38,1473.05,216.5869,167.5641
256,2023-09-19,0,366.191,335.3467,313.2814,4042.16,4504.04,6679.86,1469.36,217.0019,167.6713
257,2023-09-20,0,366.227,335.6322,313.0982,4001.75,4500.96,6648.47,1474.78,217.7547,167.9596
258,2023-09-21,0,365.919,334.2510,311.1044,3964.18,4485.67,6521.10,1428.42,216.7016,167.1376


In [2]:
for i in range(2,len(df)):
    if df.loc[i,"Date"].month != df.loc[i-1,"Date"].month:
        if df.loc[i-1,"Date"].month ==3:
            df.loc[i-1,"ME"] = 5 #Year
        elif df.loc[i-1,"Date"].month ==9:
            df.loc[i-1,"ME"] = 3 #Halfyear
        elif df.loc[i-1,"Date"].month in [6,12]:
            df.loc[i-1,"ME"] = 2 #Quater
        else:
            df.loc[i-1,"ME"] = 1
df.loc[len(df)-1,"ME"] = 5

df = df.loc[df["ME"]>=1,:].reset_index(drop=True)

for i in L:
    df.loc[i] = df[i].pct_change()

df = df.iloc[1:,:].reset_index(drop=True)


Cor = df[L].corr()
val, vec = np.linalg.eig(Cor)


vec = pd.DataFrame(vec)
for i in range(len(vec)):
    vec.iloc[i,:] = vec.iloc[i,:].apply("{:.2f}".format)


val /=  sum(val)
val = pd.DataFrame(val,columns=["val"])
for i in range(len(val)):
    val.loc[i,"accumulating"] = sum(val.loc[:i,"val"]) 

for i in range(len(val)):
    val.iloc[i,:] = val.iloc[i,:].apply("{:.2%}".format)
    
print(val)


vec.index = L
for i in range(len(Cor)):
    Cor.iloc[i,:] = Cor.iloc[i,:].apply("{:.2f}".format)
vec

       val accumulating
0   89.62%       89.62%
1   26.30%      115.91%
2  -16.30%       99.62%
3    0.27%       99.89%
4    0.08%       99.97%
5    0.05%      100.01%
6   -0.03%       99.99%
7    0.02%      100.00%
8   -0.00%      100.00%


,0,1,2,3,4,5,6,7,8
SBJ,-0.35,-0.06,-0.01,-0.08,0.31,0.43,-0.62,0.44,0.06
SBF,-0.33,-0.40,0.43,0.06,0.03,-0.35,-0.07,0.03,-0.64
SBU,-0.33,-0.41,0.43,0.13,-0.05,0.04,0.28,0.02,0.66
EQJ,-0.36,0.16,-0.28,0.60,0.21,0.25,0.48,0.16,-0.22
REJ,-0.35,-0.03,-0.05,-0.23,0.00,0.44,-0.02,-0.78,-0.12
EQF,-0.36,0.16,-0.28,0.39,-0.16,-0.51,-0.45,-0.25,0.24
REF,-0.36,0.07,-0.18,-0.29,-0.79,0.09,0.12,0.31,-0.09
USD,-0.36,0.13,-0.24,-0.57,0.46,-0.40,0.29,0.09,0.10
EUR,-0.15,0.77,0.62,-0.00,-0.00,0.01,-0.00,-0.00,-0.00
